In [ ]:

import vcf
import gzip
import subprocess
import pandas as pd
#from jupyterthemes  import jtplot

#jtplot.style(theme="onedork")

from matplotlib import pyplot as plt



In [ ]:
def condition_set(rareness,clin,pato,rev,columns,homo=None):
    '''
    rareness= binario si tomar el subset con AF segun definimos anteriormente
    homo=vector de homozigosidad minima y maxima para filtrar
    pato= vector con tipos de patogeneidad a seleccionar
    rev= status de review permitidos de la clasificacion
    clin= si usar clinvar para definir categoria
                 '''
    if clin:
    
        df_cln_info_pat=pd.read_csv("df_cln_info_pat")
        df_cln_info_pat['patho_rev'] = df_cln_info_pat['patho_rev'].apply(eval)
        

        df=df_cln_info_pat.loc[df_cln_info_pat['patho_status'].isin(pato)]
        
        #print(df.head)


        #df=df.loc[df['patho_rev'].isin(rev)]
        df=df.loc[df.patho_rev.apply(lambda x:x in rev).values]

        #print(df.head)
    if rareness:
        variants_dbfnsp=pd.read_csv("~/rare_control_variants_pops_dbNSFP4.0b2a.txt.tsv",sep="\t")
        variants_dbfnsp=variants_dbfnsp[["clinvar_id","Ensembl_geneid"]+columns+["gnomAD_exomes_controls_nhomalt"]]
        
        if homo != None:
        
            variants_dbfnsp=variants_dbfnsp.loc[(variants_dbfnsp["gnomAD_exomes_controls_nhomalt"]>homo[0])&(variants_dbfnsp["gnomAD_exomes_controls_nhomalt"]<homo[1])]
    else:
       
        reader = pd.read_csv('/home/felipe/dbNSFP4.0b2a.txt.gz',sep="\t", chunksize=100000,low_memory=False,na_values = '.',usecols=["clinvar_id","Ensembl_geneid"]+columns)
    
        lchunk=[] 
        for k,chunk in enumerate(reader):
            
            ii = chunk["clinvar_id"].isin(df["clinvar_id"])
            if sum(ii)==0:
                continue
            
            chunkf = chunk.loc[ii] 
           # print(chunkf.head)

            #chunkf = chunkf[["clinvar_id"]+columns]

            lchunk.append(chunkf)

        variants_dbfnsp = pd.concat(lchunk)

   
    if clin:
            
            df=df[['clinvar_id','auto_dominant','auto_recessive', 'x_recessive', 'x_dominant']]
        
            variants_dbfnsp=variants_dbfnsp.merge(df,how="right",on="clinvar_id",)
    
    return(variants_dbfnsp)
        

In [ ]:
# Criteria used for rare variants:
# those variants 
def extract_rareVariants_from_dbNSFP(in_f, out_f, size, freqcols , max_frequency_threshold):
    reader = pd.read_table(in_f, chunksize=size,na_values = '.')
    
    t0 = time.time()
    for k,chunk in enumerate(reader):
        minutes = (time.time()-t0)/float(60)
        if k&(k-1)==0:
            print('processed %s*%s rows. Elapsed, %s minutes',(k+1,size,minutes))
        ii = (chunk[freqcols].isna()).sum(axis=1)>0
        chunkf = chunk[ii] 
        max_freq_observed = chunkf[freqcols].max(axis=1)
        chunkf['max_freq_observed'] =  max_freq_observed
        
        # keep variants with max_freq_observed under threshold  
        # ALSO exclude those with maximum freq = 0.0000 (only for sanity)
        flag_rare_variant =  (max_freq_observed < max_frequency_threshold)&(max_freq_observed >0 )
        rares = chunkf[flag_rare_variant]

        if k==0:
            rares.to_csv(out_f, index=False,sep='\t') 
        else:
            rares.to_csv(out_f, index=False, header=False, mode='a',sep='\t')
        gc.collect()
    return(None)


In [ ]:
import re
def process_proteinwise_scores(word,fun=max):
    if pd.isna(word):
        return(None)
    if not (type(word) is str):
        return(word)
        
    averga=re.split(";",word)
    averga=set(averga)
    averga.discard(";")
    averga.discard(".")
    averga.discard(",")
    averga.discard("")
    
    if averga==set():
        return(None)
    
    mara=map(float,averga)

    av=list(mara)

    return(max(av))

    
    

In [ ]:
def get_ens(obj):
    #aver=re.split(r';',obj)
    #aver=aver[0]
    obj=str(obj)
    
    if not (';' in obj):
        return(obj)
    aver=re.search(r'(.*?);', obj).group(1)
    #k=""
    #for i in aver:
     #   k=k+i
    #return(k)
    return(aver)

In [ ]:


def forge_graph_features(df_rare_pat,target="target"):
    import igraph



    biogrid=pd.read_csv("~/BIOGRID/BIOGRID-ORGANISM-3.5.168.tab2/BIOGRID-ORGANISM-Homo_sapiens-3.5.168.tab2.txt",low_memory=False,sep="\t")



    df_biogrid=biogrid[['Entrez Gene Interactor A','Entrez Gene Interactor B']]

    tuples = [tuple(x) for x in df_biogrid.values]
    
    Gm = igraph.Graph.TupleList(tuples, directed = True, edge_attrs = ['weight'])


    dbfsnp_gen= pd.read_csv('~/dbNSFP4.0b2_gene.gz', compression='gzip',sep="\t",low_memory=False)


    dbfsnp_gen.rename(columns={"Ensembl_gene":"Ensembl_geneid"},inplace=True)



    df_ensem_to_entres=dbfsnp_gen[["Ensembl_geneid","Entrez_gene_id"]]
################################################################################
    df_rare_pat=df_rare_pat.merge(df_ensem_to_entres,how="left",on=["Ensembl_geneid"])




    Gm.vs["pathogenicity"]=[id in df_rare_pat.loc[ df_rare_pat[target]>0]["Entrez_gene_id"] for id in Gm.vs["name"]   ]


    v_patho=[vertex["name"] for vertex in Gm.vs if vertex["pathogenicity"]]


    def get_min_dist_to_patho(Vtx):
        path_to_patho=[len(path)for path in Vtx.get_shortest_paths(v_patho)]
        len_to_patho=list(set(path_to_patho))

        len_to_patho.sort()
        if(len(len_to_patho)==1):
            return(-1000)

        return(len_to_patho[1])



    Gm.vs["dist_to_patho"]=[ get_min_dist_to_patho(Vtx)  for Vtx in Gm.vs]

   
    def get_number_patho_first_neigh(Vtx):
        if(len(Gm.neighbors(Vtx))):
           return(0)
        sum(Gm.vs[list(set(Gm.neighbors(Vtx)))]["pathogenicity"])


    Gm.vs["patho_first_neigh"]=[ get_number_patho_first_neigh(Vtx)  for Vtx in Gm.vs]


    Gm.save("bio_grid_graph",format="picklez")
    
    
    
    susu=pd.DataFrame( {"Entrez_gene_id":Gm.vs["name"] , "net_dis":Gm.vs["dist_to_patho"],"net_nn":Gm.vs["patho_first_neigh"]})
    
    df_rare_pat.Entrez_gene_id.loc[df_rare_pat.Entrez_gene_id.isna()]=-2
    
    susu=df_rare_pat.merge(susu,how="left",on=["Entrez_gene_id"])
    return(susu)

    
    

    

In [ ]:
df_cln_info_pat=pd.read_csv("df_cln_info_pat",low_memory=False)